# LSST Standard Time
Determine the latest sunrise (UTC) at the LSST location in Chile and round to the next closest hour. This is LSST Standard Time and the point at which date rollovers happen.

In [1]:
from datetime import datetime, timedelta

from astropy.time import Time
from ephem import Observer, Sun
import numpy as np
import pandas as pd
import pytz

In [2]:
lsst_latitude = "-30:14:38.4"
lsst_longitude = "-70:44:57.84"
lsst_height = 2650.0  # meters
lsst_pressure = 750.0  # millibars

In [3]:
lsst_sunrise = Observer()
lsst_sunrise.long = lsst_longitude
lsst_sunrise.lat = lsst_latitude
lsst_sunrise.elevation = lsst_height
lsst_sunrise.horizon = "-0:34"
lsst_sunrise.pressure = lsst_pressure

In [4]:
chile_tz = pytz.timezone('Chile/Continental')
sun = Sun()
full_format = "%Y/%m/%d %H:%M %z"

In [5]:
original_date = datetime(2018, 1, 1)
end_date = datetime(2019, 1, 1)
days = (end_date - original_date).days
date = original_date

column_names = ['Datetime', 'Hour', 'Minute']

df_rows = []

for i in range(days):
    local_time = chile_tz.localize(date)
    atime = Time(local_time, scale='utc')
    dtd = atime.datetime
    lsst_sunrise.date = dtd
    sunrise = lsst_sunrise.next_rising(sun).datetime()
    df_rows.append(pd.DataFrame([[sunrise, sunrise.hour, sunrise.minute]], columns=column_names))
    date += timedelta(1)
df = pd.concat(df_rows, ignore_index=True)

In [6]:
max_hour = np.max(df['Hour'])
max_minute = np.max(df['Minute'][df['Hour'] == max_hour])
print("Time of latest sunrise: {}:{} UTC".format(max_hour, max_minute))
rollover_hour = max_hour + 1
print("LSST Standard Time: {}:00 UTC".format(rollover_hour))

Time of latest sunrise: 11:37 UTC
LSST Standard Time: 12:00 UTC
